## Cost Update

Test methods for updating the global costmap from local observations

In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import cv2 as cv
import GPy

from terrain_nerf.global_planner import GlobalPlanner
from terrain_nerf.utils import rgb2gray
from terrain_nerf.feature_map import FeatureMap, CostMap
from terrain_nerf.autonav import AutoNav

# autoreload
%load_ext autoreload
%autoreload 2

### Parameters and Setup

In [ ]:
UNREAL_PLAYER_START = np.array([-117252.054688, 264463.03125, 25148.908203])
UNREAL_GOAL = np.array([-83250.0, 258070.0, 24860.0])

GOAL_POS = (UNREAL_GOAL - UNREAL_PLAYER_START)[:2] / 100.0

global_img = cv.imread('../data/airsim/images/test_scenario_3.png')
global_img = global_img[::2, ::2, :]
start_px = (138, 141)
goal_px = (78, 493)

### Global costmap and planner

In [ ]:
costmap_data = np.load('../data/airsim/costmap.npz')
costmap = CostMap(costmap_data['mat'], costmap_data['clusters'], costmap_data['vals'])

feat_map = FeatureMap(global_img, start_px, goal_px, UNREAL_PLAYER_START, UNREAL_GOAL)
global_planner = GlobalPlanner(costmap, feat_map, goal_px)

init_path = global_planner.replan(np.zeros(3))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(global_img)
im = global_planner.plot(ax[1])
# plt.colorbar(im, ax=ax[1])
plt.show()

### Local observation

In [ ]:
depth = np.load('../data/airsim/images/depth_1688174586.6823223.npy')
pose = np.array([169.813, -23.864, -0.104])  # x, y, yaw

depth_vis = depth.copy()
depth_vis[depth_vis > 20] = 20.0

autonav = AutoNav(np.zeros(2))
cost_vals = autonav.update_costmap(pose, depth)
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(depth_vis)
im = autonav.plot_costmap(ax[1])
plt.colorbar(im)

In [ ]:
cvs = np.array(cost_vals)
plt.scatter(cvs[:, 1], cvs[:, 0], c=cvs[:, 2], cmap='viridis')

### GP-based update

In [ ]:
GPy.plotting.change_plotting_library('plotly')

In [ ]:
X = np.random.uniform(-3.,3.,(20,1))
Y = np.sin(X) + np.random.randn(20,1)*0.05

kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(X,Y,kernel)

In [ ]:
m.optimize()

In [ ]:
fig = m.plot()
GPy.plotting.show(fig, filename='basic_gp_regression_notebook')

In [ ]:

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import GPy
import numpy as np
np.random.seed(101)

In [ ]:
N = 50
noise_var = 0.05

X = np.linspace(0,10,50)[:,None]
k = GPy.kern.RBF(1)
y = np.random.multivariate_normal(np.zeros(N),k.K(X)+np.eye(N)*np.sqrt(noise_var)).reshape(-1,1)

In [ ]:
m_full = GPy.models.GPRegression(X,y)
m_full.optimize('bfgs')
m_full.plot()
print(m_full)